In [1]:
# 설치 진행
!pip install selenium

In [1]:
# 1.webdriver - 웹 통제 라이브러리
# 2. Keys = 컴퓨터용 키보드
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
import os 
#파일시스템을 위한 라이브러리 ex)파일,폴더를 생성,삭제,존재여부

from urllib.request import urlretrieve 
# 이미지경로를 파일로 저장

In [5]:
# 폴더 생성 make dir
if not os.path.isdir('./img') : # 이미 경로상에서 존재하는지 확인
        os.mkdir('./img') 

In [6]:
if not os.path.isdir('./processPhoto') : # 이미 경로상에서 존재하는지 확인
        os.mkdir('./processPhoto')

In [3]:
# 크롬 실행
driver=wb.Chrome() 
driver.get("https://terms.naver.com/list.naver?cid=42785&categoryId=42788&so=st3.asc&viewType=&categoryType=") 
# url값으로 페이지이동

In [12]:
# 안으로 들어가기
link=driver.find_elements_by_css_selector('span.mask')

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  


In [4]:
title_list=[] # 제목
main_photo_link=[]
ingr_list=[]
ingredients_Mainlist=[] #주재료
ingredients_Sublist=[] #부재료

time_list=[] #요리 소요시간
howMany_list=[] #분량
calorie_list=[] #칼로리

cookProcess_list=[] #요리과정
processPhoto_list=[] #요리과정의 사진

#영양성분
carbohydrate_list=[]
protein_list=[]
fat_list=[]
calcium_list=[]
sodium_list=[]


num=1


for page in range(24):
    print(page+1,"번 페이지")
    driver.get(f'https://terms.naver.com/list.naver?cid=42785&categoryId=42788&so=st3.asc&viewType=&categoryType=&page={page+1}')
    
    
    link=driver.find_elements_by_css_selector('span.mask')
    print(len(link))
    for i in range(len(link)-1):
        time.sleep(0.1) 
        link=driver.find_elements_by_css_selector('span.mask')
        link[i].click()
        time.sleep(0.1) 
        soup = bs(driver.page_source,'lxml')

        #음식 이름
        title = soup.select_one('h2.headword')
        title_list.append(title.get_text())

        #메인 이미지
        main_photo=soup.select_one('img#innerImage0')
        main_photo_link.append(main_photo['data-src'])
        #urlretrieve(main_photo['data-src'],f'./img/main_photo{i+1}.jpg')


        #주재료, 부재료
        ingr=soup.select_one('h4#TABLE_OF_CONTENT2 ~ p.txt')
        ingr=ingr.get_text()
        ingr_list=ingr.split('· ')
        
        
        
        temp_ingr=[None,None]
        for i in ingr_list:
            list2=i.split(' : ')
            if('주재료' in list2[0]):
                temp_ingr[0]=list2[1].replace('\n','')
            elif('부재료' in list2[0]):
                temp_ingr[1]=list2[1].replace('\n','')
        
        
        ingredients_Mainlist.append(temp_ingr[0])
        ingredients_Sublist.append(temp_ingr[1])
     
        #
        info=soup.select('p.txt')    


        # 기본정보 - 조리시간, 분량,분량
        info[2].text
        info_list=info[2].text.replace('· ','')
        info_list=info_list.split('\n')

                

        
        temp = [None,None,None]
        for i in info_list:
            list1=i.split(' : ')
            if('조리시간' in list1[0]):
                temp[0] = list1[1]
            elif('분량' in list1[0]):
                temp[1] = list1[1]
            elif('칼로리' in list1[0]):
                temp[2] = list1[1]
        
        time_list.append(temp[0]) #요리 소요시간
        howMany_list.append(temp[1]) #분량
        calorie_list.append(temp[2])
          
        # 조리 순서
        #한번에 붙여야되나?
        recipe_cnt=1
        recipe=''
        for i in info[3:]:
            if('0' in i.text):
                recipe+=i.text+'\n'
                recipe_cnt+=1
        cookProcess_list.append(recipe)

        # 조리과정 사진

        # 레시피별로 과정마다의 사진이 따로 폴더에 저장.
        #if not os.path.isdir(f'./processPhoto/{num}_process') : # 이미 경로상에서 존재하는지 확인
        #    os.mkdir(f'./processPhoto/{num}_process')
        
        #사진링크+사진링크+사진링크??????
        img_list=soup.select('img')
        pp=''
        for i in range(4,3+recipe_cnt):
            try:
                pp+=img_list[i]['origin_src']+','
                #urlretrieve(img_list[i]['origin_src'],f'./processPhoto/{num}_process/{i-3}.jpg')
            except:
                pp+=','
        processPhoto_list.append(pp)
        num+=1



        #영양성분 - 탄수화물, 단백질, 지방, 칼슘, 나트륨
        nu=soup.select_one('div.box_tbl')

        temp_nu=[None,None,None,None,None]
        try:
            if(nu is not None):
                        temp_nu[0]=nu('span')[1].text
                        temp_nu[1]=nu('span')[2].text
                        temp_nu[2]=nu('span')[3].text
                        temp_nu[3]=nu('span')[4].text
                        temp_nu[4]=nu('span')[6].text

        except:
            print(f'{page+1}페이지의 {num}에서 문제 발생')
        
        carbohydrate_list.append(temp_nu[0])
        protein_list.append(temp_nu[1])
        fat_list.append(temp_nu[2])
        calcium_list.append(temp_nu[3])
        sodium_list.append(temp_nu[4])
            
        driver.back()
        time.sleep(0.1)

1 번 페이지


C:\Users\smhrd\AppData\Local\Temp/ipykernel_2968/3865931839.py:31: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  link=driver.find_elements_by_css_selector('span.mask')
C:\Users\smhrd\AppData\Local\Temp/ipykernel_2968/3865931839.py:35: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  link=driver.find_elements_by_css_selector('span.mask')


16
2 번 페이지
16
3 번 페이지
16
4 번 페이지
16
5 번 페이지
16
6 번 페이지
16
6페이지의 86에서 문제 발생
7 번 페이지
16
8 번 페이지
16
9 번 페이지
16
10 번 페이지
16
11 번 페이지
16
12 번 페이지
16
13 번 페이지
16
14 번 페이지
16
14페이지의 202에서 문제 발생
15 번 페이지
16
16 번 페이지
16
17 번 페이지
16
18 번 페이지
16
19 번 페이지
16
20 번 페이지
16
21 번 페이지
16
22 번 페이지
16
23 번 페이지
16
24 번 페이지
11


In [144]:
processPhoto_list

['https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060149039_KKD4TNKUB.jpg/inst_13_i2.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060153403_E3W95WIEI.jpg/inst_13_i3.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060153749_CUQ44WCWM.jpg/inst_13_i4.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060215315_G4AUO27TW.jpg/inst_13_i5.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060219158_18LX7MF9X.jpg/inst_13_i6.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060223448_XBNCDXSHM.jpg/inst_13_i7.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060858727_PJ7KEZP86.jpg/inst_31_i2.jpg?type=m4500_4500_fst_n&wm=Y',
 'https://dbscthumb-phinf.pstatic.net/2756_000_1/20131109060900183_RI88JLNQC.jpg/inst_31_i3.jpg?type=m4500_4500_fst_n&wm=Y',


In [5]:
print(len(title_list))
print(len(main_photo_link))
print(len(ingredients_Mainlist))
print(len(ingredients_Sublist))
print(len(time_list))
print(len(calorie_list))
print(len(cookProcess_list))
print(len(carbohydrate_list))
print(len(protein_list))
print(len(fat_list))
print(len(calcium_list))
print(len(sodium_list))

print(len(processPhoto_list))

355
355
355
355
355
355
355
355
355
355
355
355
355


In [6]:
recipe_Infomaiton={'음식 이름':title_list,
                   '메인 사진':main_photo_link,
            '주재료':ingredients_Mainlist,'부재료':ingredients_Sublist,
           '소요시간':time_list,'분량':howMany_list,'칼로리':calorie_list,
            '요리과정':cookProcess_list,
                   '과정 사진':processPhoto_list,
            '탄수화물':carbohydrate_list,'단백질':protein_list,
           '지방':fat_list,'칼슘':calcium_list,'나트륨':sodium_list
           }

In [7]:
df=pd.DataFrame(recipe_Infomaiton)

In [9]:
df.to_csv('네이버_레시피카드_원본.csv', encoding='utf-8-sig')

In [91]:
time_list

['30분',
 None,
 None,
 None,
 '20분',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '40분',
 None,
 None,
 None,
 '45분',
 None,
 None,
 None,
 '40분',
 None,
 None,
 None,
 '45분',
 None,
 None,
 None,
 '20분',
 None,
 None,
 None,
 None,
 None,
 None,
 '25분',
 None,
 None,
 None,
 None,
 None,
 None,
 '30분분량',
 '35분',
 None,
 None,
 None]

In [96]:
info_list

['조리시간 : 35분', '분량 : 4인분 기준', '칼로리 : 783kcal (1인분)', '']